# Лемматизация и очистка от контекстных стоп-слов

In [1]:
import pandas as pd
import numpy as np
import os
import re
import glob

from tqdm.notebook import tqdm

np.random.seed(10)

UNIVS = ['АлтГУ', 'АлтГТУ', 'ВолГУ', 'ИМСИТ']
CURRPATH = os.getcwd()
DATAPATH = '\\'.join(CURRPATH.split('\\')[:-1]) + '\\new-data'
FILESPATH = DATAPATH + '\\new-files'

# Документы, которые невозможно адекватно парсить:
IGNOREFILES = [
    '\\ИМСИТ\\педагогическое образование история и право 3++\\сопоставительная германистика.txt',
    '\\ИМСИТ\\педагогическое образование англ нем 3++\\культурология.txt',
    '\\ИМСИТ\\педагогическое образование англ нем 3++\\история немецкого языка.txt',
    '\\ИМСИТ\\педагогическое образование изобразительное искусство и информатика\\основы проектной деятельности.txt',
    '\\ИМСИТ\\педагогическое образование история и право 3++\\методика обучения немецкий язык.txt',
    '\\ИМСИТ\\педагогическое образование история и право 3++\\основы проектной деятельности.txt',
    '\\ИМСИТ\\педагогическое образование история и право 3++\\практический курс английского языка.txt',
    '\\ИМСИТ\\прикладная информатика\\системы управления хранилищами данных.txt',
    '\\ИМСИТ\\прикладная информатика\\теория вероятностей и математическая статистика.txt',
    '\\ИМСИТ\\прикладная информатика\\физическая культура и спорт общая физическая подготовка.txt',
    '\\ИМСИТ\\реклама и связи с общественностью 3++\\практикум технологиии эффективного общения.txt',
    '\\АлтГУ\\юриспруденция общеправовой\\конституционное право.txt',
    '\\АлтГТУ\\цифровая экономика\\проектирование информационных систем.txt',
]

# Образовательные направления -- крайне специфичные и сложнокластеризуемые
# Для таких направлений нужно много данных
IGNOREFOLDERS = [
    'гостиничное дело',
    'графический дизайн',
]

## Считывание данных

In [2]:
def read_to_df(folder: str, ignorefiles=IGNOREFILES, ignorefolders=IGNOREFOLDERS):
    df = pd.DataFrame(columns=['univ', 'prog', 'name', 'text'])
    for file in tqdm(glob.glob('{}/*/*/*.txt'.format(folder))):
        splitted = file.split('\\')
        name = splitted[-1][:-4]
        u = splitted[-3]
        op = splitted[-2]
        
        path = '\\' + u + '\\' + op + '\\' + splitted[-1]
        if  path in ignorefiles or \
                op in ignorefolders or \
                'практик' in splitted[-1] or \
                'введение в специальность' in splitted[-1] or \
                'введение в професси' in splitted[-1]:
            continue
        
        try:
            text = open(file, 'r').read()
        except Exception as e:
            print(e)
            print(file)
            text = open(file, 'r', encoding='utf-8').read()
        
        df = df.append({'univ':u, 'prog':op, 'name':name, 'text':text.lower()}, ignore_index=True)
    return df

In [3]:
df = read_to_df(FILESPATH)
len(df)

  0%|          | 0/3169 [00:00<?, ?it/s]

3009

## Удаление дубликатов

In [4]:
df.drop_duplicates(subset='text', inplace=True)
len(df)

2464

## Лемматизация

In [5]:
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

from pymystem3 import Mystem
ms = Mystem()

a = ord('а')
DEFAULT_STOP = set(stopwords.words('russian') + [chr(i) for i in range(a, a + 32)] + [_ for _ in range(10)])

In [6]:
def lemmatize(text, regs=None):
    lemmas = ms.lemmatize(text)
    text = ''.join(lemmas).rstrip('\n')
    if regs is not None:
        for reg in regs:
            text = re.sub(reg, '', text)
    return text


def process_text(text, f_lemmatizer=lemmatize, wstop=DEFAULT_STOP,
                 lemm_regs=None, other_regs=None, counter=None):
    
    if f_lemmatizer is not None:
        text = f_lemmatizer(text, lemm_regs)
    
    if other_regs is not None:
        for reg in other_regs:
            text = re.sub(reg, '', text, flags=re.IGNORECASE)
              
    text = [word for word in word_tokenize(text) if word not in wstop and str.isalpha(word[0])]
    if counter is not None:
        counter.update(text)
    return ' '.join(text)

In [7]:
import time
fd_counter = FreqDist()

start = time.time()
df.text = df.text.apply(process_text, counter=fd_counter)
print('Time spent: {} s.'.format(time.time() - start))

Time spent: 2226.985454559326 s.


In [8]:
len(fd_counter)

18224

In [9]:
import json
jsonStr = json.dumps(dict(fd_counter.most_common()), ensure_ascii=False)
file = open('JsonCounter.json', 'w')
file.write(jsonStr)
file.close()

Установим пороги на число вхождений: верхний 2100, нижний -- 1, так как среди таких слов много опечаток по вине составителей документов. В этом случае некоторые документы могут потерять суть, но этим стоит пожертвовать в угоду других документов. Отдельно вернем слова иностранных языков и некоторые другие термины. Удалим слова, длина которых не более двух.

In [10]:
file_ = open('saved.txt', 'r', encoding='utf-16')
saved_words = file_.read().replace('\{', '').replace('\}', '').replace(': 1', '').replace('\"', '').split(',')
file_.close()

In [11]:
d = fd_counter
words_most = []
words_least = []
other_words = []
most = 400
least = 1
for w in d.keys():
    if d[w] >= most:
        words_most += [w]
    elif d[w] <= least:
        words_least += [w]
    elif len(w) < 3:
        other_words += [w]

Также удалим некоторые контекстные слова, которые в теории могут мешать.

In [12]:
context_stop = [
    'определять', 'выявлять', 'актуальный', 'уп', 'опоп', 'пример', 'понимание',
    'понимать', 'знать', 'знание', 'учитывать', 'дв', 'владение', 'описание', 'проводить',
    'весь', 'вся', 'все', 'проектный', 'который', 'особый', 'ознакомление', 'the', 'опк',
    'специалист', 'семестр', 'первый', 'второй', 'третий', 'четвертый', 'пятый', 'знакомство',
    'готовность', 'модуль', 'задание', 'др', 'иметься', 'единица', 'соответствующий', 'часы',
    'работник', 'задание', 'обосновывать', 'учитывать', 'формат', 'наиболее', 'наименее',
    'формулировать', 'заключение', 'научать', 'зачет', 'познакомить', 'a', 'выделять',
    'участвовать', 'экзамен', 'оформлять', 'соблюдать', 'делать', 'обоснованный', 'лекция',
    'урок', 'содержаться', 'содержание', 'каждый', 'текущий', 'это', 'существующий', 'учащийся',
    'выпускной', 'учебник', 'интернет-ресурс', 'ожидать', 'заочный', 'очный', 'го', 'обзор',
    'раздел', 'результат', 'освоение', 'применение', 'использование', 'изучение', 'практика',
    'представление', 'владеть', 'знать', 'уметь', 'применять', 'понятие', 'формирование',
    'создание', 'развитие', 'получать', 'иметь', 'опыт', 'студент', 'изучение', 'структура', 'умение',
    'использовать', 'общий', 'организация', 'опк', 'oпк', 'ок', 'принцип', 'назначение', 'цель',
    'способный', 'определение', 'построение', 'вид', 'проект', 'особенность', 'стр', 'стp', 'cтp',
    'б1', 'выпускник', 'иной', 'развивать', 'текущий', 'важный', 'позволять',  'направление',
    'бакалавр', 'находить', 'обладать', 'тема', 'ых', 'ый', 'подготовка', 'оформление', 'сосредоточивать',
    'самостоятельно', 'профиль', 'специалист', 'осваивать', 'общепрофессиональный', 'a', 'разный',
    'решать', 'июль', 'ока', 'oка', 'окa', 'квалификационный', 'осуществление', 'отчет', 'весь',
    'контрольный', 'овладение', 'самостоятельный', 'давать', 'работать', 'выбирать', 'свой', 'курс',
    'формировать', 'помещениe', 'наименование', 'также', 'сформированность', 'обладать', 'небольшой',
    'большой', 'малый', 'тео-рий', 'интер-нет', 'пять', 'организа-ции', 'сист', 'ми', 'ч2', 'ч1', 'ч3',
    'соци-альный', 'подготовка', 'пожготовка', 'весьма', 'куультура', 'политическои', 'вспомогатель-ных',
    'элька-тромагнетизм', 'вещий-ство', 'воспа', 'эксплуотация', 'социаль-ного', 'подго-товка',
    'кон-цепция', 'особенностмевать', 'прочее', 'гражданин-ского', 'требовать', 'воспринимать', 'включая',
    'опираться', 'ук', 'методология', 'методологический', 'инструмент', 'составлять', 'влияние', 'публич-ную',
    'полити-ческий', 'поставлять-ных', 'исследовый-нию', 'транс-формация', 'формулиро-вания', 'самостоятельно-го',
    'социологич-ские', 'иссле-дование', 'норматив-ной', 'функ-циональный', 'расти-сийский', 'лредоставление',
    'характа-ризовать', 'публич-ную', 'пройка-тирование', 'эко-номический', 'эконо-метрический', 
    'экономет-рический', 'эконометриче-ского', 'изученииособенность', 'знаия',
]


In [13]:
upd_wstop = set(words_most + words_least + other_words + context_stop) - set(saved_words)
len(upd_wstop), set(saved_words) in upd_wstop

(8021, False)

In [14]:
fd_counter_trunc = FreqDist()
df_trunc = df.copy()

In [15]:
start = time.time()
df_trunc.text = df_trunc.text.apply(process_text, f_lemmatizer=None, wstop=upd_wstop, counter=fd_counter_trunc)
print('Time spent: {} s.'.format(time.time() - start))

Time spent: 2.8519551753997803 s.


In [16]:
len(list(dict(fd_counter_trunc)))

10217

In [17]:
jsonStr_trunc = json.dumps(dict(fd_counter_trunc.most_common()), ensure_ascii=False)
file = open('JsonCounter_trunc.json', 'w')
file.write(jsonStr_trunc)
file.close()

## Запись

In [18]:
def rewrite(folder: str, df):
    for row in tqdm(df.iterrows()):
        r = row[1]
        path = folder + '\\' + r['univ'] + '\\' + r['prog']
        try:
            os.makedirs(path)
        except FileExistsError:
            pass
        
        filename = path + '\\' + r['name'] + '.txt'

        file_ = open(filename, 'w', encoding='ansi')
        try:
            file_.write(r['text'])
        except Exception as e:
            print('FATAL:', e)
            print(row[0], filename)
            file_.close() 
            break
        file_.close()

In [19]:
rewrite(DATAPATH + '\\new-files-lemm', df_trunc)

0it [00:00, ?it/s]